In [ ]:
!pip install transformers datasets accelerate torch evaluate rouge-score evaluate

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.4 MB/s eta 0:00:00
   

In [ ]:
import torch
from datasets import load_dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer, DataCollatorForSeq2Seq


In [ ]:
from datasets import load_dataset

# Load the datasets
train_dataset = load_dataset('csv', data_files='/content/drive/MyDrive/BTP/BTP_LLM-20241106T104614Z-001/BTP_LLM/train_data.csv')['train']
val_dataset = load_dataset('csv', data_files='/content/drive/MyDrive/BTP/BTP_LLM-20241106T104614Z-001/BTP_LLM/test_data.csv')['train']
test_dataset = load_dataset('csv', data_files='/content/drive/MyDrive/BTP/BTP_LLM-20241106T104614Z-001/BTP_LLM/val_data.csv')['train']

train_dataset = train_dataset.shuffle(seed=42).select(range(15000))
val_dataset = val_dataset.shuffle(seed=42).select(range(1000))
test_dataset = test_dataset.shuffle(seed=42).select(range(5000))

# Inspecting a sample data to check structure
print(train_dataset[0])
print(test_dataset[0])
print(val_dataset[0])


print("Number of samples in train dataset:", len(train_dataset))
print("Number of samples in test dataset:", len(test_dataset))
print("Number of samples in validation dataset:", len(val_dataset))



Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

{'recipe_name': 'Big Sky Beef', 'ingredients': '["1 lb. Chuckwagon Ground Beef", "8 oz. can tomato sauce", "1/2 c. onions, chopped", "16 oz. can kidney beans", "1 pkg. taco seasoning mix", "1 c. water", "1-1/2 c. shredded cheddar cheese", "2 c. Fritos Corn Chips"]', 'instructions': '["Brown the ground beef and onion; drain. Mix in the taco seasoning mix and water, bring to a boil. Reduce heat. Add tomato sauce and kidney beans; simmer for 10 minutes. Spread 1 cup of the corn chips in baking dish. Layer with half of the meat mixture. Next layer with half of the cheddar cheese. Spread second cup of corn chips and repeat layers. Cover and bake at 350 degrees for 30 minutes. Serve with salsa and sour cream."]'}
{'recipe_name': 'Applesauce Fruit Cake', 'ingredients': '["1 c. butter", "1 c. white sugar", "1 c. brown sugar", "2 eggs", "4 c. sifted flour", "2 c. raisins", "1/2 c. chopped candied cherries", "1 c. candied pineapple", "1 c. candied fruit mix", "1 c. nut meats", "1 tsp. soda", "2 

In [ ]:
from transformers import AutoTokenizer

def preprocess_function(examples):
    inputs = [f"Generate recipe instructions for: {name}\nIngredients: {ing}"
              for name, ing in zip(examples['recipe_name'], examples['ingredients'])]
    targets = examples["instructions"]
    return {"inputs": inputs, "targets": targets}

# tokenized_datasets = dataset.map(preprocess_function, batched=True)


In [ ]:
train_data = train_dataset.map(preprocess_function, batched=True)
val_data = val_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)

# Tokenization function
def tokenize_function(examples):
    model_inputs = tokenizer(examples["inputs"], max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(examples["targets"], max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenizing datasets
train_tokenized = train_data.map(tokenize_function, batched=True)
val_tokenized = val_data.map(tokenize_function, batched=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"


In [ ]:
import torch
from transformers import AutoModelForSeq2SeqLM, TrainingArguments, Trainer, EarlyStoppingCallback, TrainerCallback
from transformers import AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")

# import torch
# from transformers import AutoModelForSeq2SeqLM, TrainingArguments, Trainer, EarlyStoppingCallback, TrainerCallback
# from transformers import AutoTokenizer

# # Load model and tokenizer
# model_name = "t5-small"  # Change this if using another model
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Define training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/BTP/T5_small/t5_new_recipe_fine_tuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_steps=165,  # Save every 0.1 epoch
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    weight_decay=0.01,
    num_train_epochs=10,
    save_total_limit=3,  # Keep only last 3 checkpoints
    load_best_model_at_end=True,
    logging_dir="./logs",
    remove_unused_columns=False,
    logging_steps=50,
    metric_for_best_model="eval_loss",
    greater_is_better=False  # Lower validation loss is better
)

# Custom callback to save the best model
class SaveBestModelCallback(TrainerCallback):
    def __init__(self, save_path):
        self.best_loss = float('inf')
        self.save_path = save_path

    def on_evaluate(self, args, state, control, metrics, **kwargs):
        val_loss = metrics.get('eval_loss')
        if val_loss is not None:
            print(f"Validation Loss: {val_loss}")

            if val_loss < self.best_loss:
                self.best_loss = val_loss
                print(f"New best model found! Saving to {self.save_path}")
                model.save_pretrained(self.save_path)
                tokenizer.save_pretrained(self.save_path)

# Define paths
best_model_path = "/content/drive/MyDrive/BTP/T5_small/best_model"

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[
        SaveBestModelCallback(best_model_path),
        EarlyStoppingCallback(early_stopping_patience=3)  # Stops training if no improvement in 3 evaluations
    ]
)

# Train the model
trainer.train()

# Save the final model
final_model_path = "/content/drive/MyDrive/BTP/T5_small/final_model"
model.save_pretrained(final_model_path)
tokenizer.save_pretrained(final_model_path)
print(f"Final model saved at {final_model_path}")


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


TypeError: can only join an iterable

In [ ]:

# Load the best model manually if needed
from transformers import T5ForConditionalGeneration

best_model = T5ForConditionalGeneration.from_pretrained(best_model_dir)
best_model.save_pretrained("best_t5_recipe")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install rouge-score
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.8 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
import evaluate
import ast

# Define the path to your fine-tuned model checkpoint
checkpoint_path = "/content/drive/MyDrive/BTP/T5_small/t5_recipe_fine_tuned/checkpoint-1650"

# Load tokenizer and model
tokenizer = T5Tokenizer.from_pretrained(checkpoint_path)
model = T5ForConditionalGeneration.from_pretrained(checkpoint_path)

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Load evaluation metrics
rouge = evaluate.load("rouge")
bleu = evaluate.load("bleu")

# Function to generate recipe instructions
def generate_recipe(ingredients, max_length=200):
    input_text = f"Generate recipe instructions for ingredients: {ingredients}"

    # Tokenize input
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move to GPU

    # Generate instructions
    with torch.no_grad():
        output_tokens = model.generate(**inputs, max_length=max_length)

    # Decode output
    generated_recipe = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    return generated_recipe

# Sample test case from test dataset
test_sample = {
    'recipe_name': 'Applesauce Fruit Cake',
    'ingredients': '["1 c. butter", "1 c. white sugar", "1 c. brown sugar", "2 eggs", "4 c. sifted flour", "2 c. raisins", "1/2 c. chopped candied cherries", "1 c. candied pineapple", "1 c. candied fruit mix", "1 c. nut meats", "1 tsp. soda", "2 tsp. baking powder", "3 tsp. cinnamon", "1 tsp. ground cloves", "1 tsp. ground ginger", "1 tsp. vanilla"]',
    'instructions': '["Mix fruits, nuts, spices and baking powder in flour.", "Mix soda in applesauce.", "Cream margarine, sugar and eggs.", "Stir in flour mixture with applesauce.", "Grease angel food pan or loaf pan.", "Pour mixture into pans.", "Bake in a 325° oven for 1 hour.", "Test for doneness."]'
}

# Get ingredients and ground-truth instructions
ingredients = test_sample['ingredients']
reference_instructions = ast.literal_eval(test_sample['instructions'])  # Convert string to list

# Generate recipe instructions
generated_instructions = generate_recipe(ingredients)
print("generated_instructions ",generated_instructions)
print("original_instructions ",test_sample['instructions'])
# Compute ROUGE score
rouge_score = rouge.compute(predictions=[generated_instructions], references=[" ".join(reference_instructions)])
print("\nROUGE Score:", rouge_score)

# Compute BLEU score
reference_tokens = [[ref.split()] for ref in reference_instructions]  # BLEU expects nested list format
generated_tokens = generated_instructions.split()

bleu_score = bleu.compute(predictions=[generated_tokens], references=reference_tokens)
print("\nBLEU Score:", bleu_score)


generated_instructions  ["In a large bowl, combine butter, white sugar, brown sugar, eggs, flour, raisins, cherries, pineapple, fruit mix, nut meats, soda, baking powder, cinnamon, ginger and ginger.", "Add remaining ingredients.", "Add remaining ingredients.", "Add remaining ingredients.", "Add remaining ingredients.", "Add remaining ingredients.", "Add remaining ingredients.", "Add remaining ingredients.", "Add remaining ingredients.", "Add remaining ingredients.", "Add remaining ingredients.", "Add remaining ingredients.", "Bake at 350u00b0 for 45 minutes."]
original_instructions  ["Mix fruits, nuts, spices and baking powder in flour.", "Mix soda in applesauce.", "Cream margarine, sugar and eggs.", "Stir in flour mixture with applesauce.", "Grease angel food pan or loaf pan.", "Pour mixture into pans.", "Bake in a 325° oven for 1 hour.", "Test for doneness."]

ROUGE Score: {'rouge1': 0.2162162162162162, 'rouge2': 0.03669724770642201, 'rougeL': 0.12612612612612614, 'rougeLsum': 0.126

ValueError: Predictions and/or references don't match the expected format.
Expected format:
Feature option 0: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}
Feature option 1: {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')},
Input predictions: ['["In', 'a', 'large', ..., 'for', '45', 'minutes."]'],
Input references: [['Mix', 'fruits,', 'nuts,', 'spices', 'and', 'baking', 'powder', 'in', 'flour.']]